#**Ciencia de Datos Aplicada MINE-4101 - TALLER 3**

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/University_of_Los_Andes_logo.svg/1200px-University_of_Los_Andes_logo.svg.png" style="width:50px;"> 

Univerisdad de los Andes  
Realizado por: 

  Oscar Fernando Bosigas Puerto – 202220008 - o.bosigas

  Yeimy Andrea Cano M – 202213304 - y.cano

  David Polania Mejia - 202213328 - d.polaniam

# **0. Importación de librerias de trabajo**

Primero se procede a hacer la importación de librerías para poder trabajar con los datos y poder realizar el análisis.

In [1]:
#!pip install --upgrade pandas-profiling

In [2]:
import numpy as np
import pandas as pd

from pandas_profiling import ProfileReport
import seaborn as sns
from scipy import stats
import scipy
import statsmodels.api as sm

#Entrenamiento del modelo
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures , OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression,Lasso, Ridge
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.dpi'] = 110    
import pylev

# **Entendimiento del Negocio**




Una empresa del sector farmacéutico desea utilizar la información que posee de sus empleados para estimar el nivel de satisfacción laboral que puedan tener en algún momento y usarlo como insumo para detectar posibles fugas de personal. 

El CEO de la farmacéutica le ha pedido a su equipo de científicos de datos que construyan un modelo para tal fin y le proporcionen un conjunto de estrategias que permitan aumentar el nivel de satisfacción laboral de sus empleados y evitar futuras fugas de personal.

## **a. Objetivos del negocio y situación actual.** 

*   Generar un modelo que permita detectar posibles fugas de personal a partir de los datos de satifacción laboral.


# **Entendimiento de los datos**

A continuación el diccionario de datos entregado por el negocio:

|Modelo|Description|
|---|---|
|Age| Age of the employee|
|Attrition| employee attrition|
|BusinessTravel| how frequently an employee travels for business purpose|
|DailyRate| Daily wage of an employee|
|Department| Employee department|
|DistanceFromHome| Distance from home to office in KM's|
|Education| Qualification of employee (masked, higher is better)|
|EducationField| Stream of Education|
|EmployeeCount| EmployeeCount|
|EmployeeNumber| employee number|
|EnvironmentSatisfaction| Environment (higher is better)|
|Gender| Gender of employee|
|HourlyRate| employee hourly rate|
|JobInvolvement| Job involvement (higher is better)|
|JobLevel| level of Job (higher is more important)|
|JobRole| job role of an employee|
|JobSatisfaction| if employee is satisfied?|
|MaritalStatus| employee is married or not|
|MonthlyIncome| income of an employee|
|MonthlyRate| monthly rate of an employee|
|NumCompaniesWorked| number of companies worked for|
|Over18| age over 18|
|OverTime| employee works over time|
|PercentSalaryHike| salary hike|
|PerformanceRating| performance rate|
|RelationshipSatisfaction| Relationship satisfaction|
|StandardHours| per week standard work hours|
|StockOptionLevel| company stock option level|
|TotalWorkingYears| total working years|
|TrainingTimesLastYears| Training time|
|WorkLifeBalance| Work life balance|
|YearsAtCompany| total years at current company|
|YearsInCurrentRole| total years in current role|
|YearsSinceLastPromotion| years since last promotion|
|YearsWithCurrManager| Years worked under current manager|

De acuero a la descripción de las columnas, la variable objetivo es *'Attrition'*

## **a. Adquisición e integración de datos.**

El archivo de datos está como archivo de CSV, separado por ';'. El archivo se deja en github para poder leer los datos desde ahi:

In [3]:
# Loading data
url = 'https://raw.githubusercontent.com/yacanom/CDA_MINE-4101_repo/main/Taller%203/Data_Taller_3.csv'
data = pd.read_csv(url, sep=';', decimal=",")

Se revisa la estructura de los datos

In [4]:
data.head(10)

,JobSatisfaction,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,NewEmployee
0,4.0,41.0,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,...,80.0,0,8,0,1,6,4,0,5,0
1,2.0,49.0,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,...,80.0,1,10,3,3,10,7,1,7,0
2,3.0,37.0,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,...,80.0,0,7,3,3,0,0,0,0,0
3,3.0,33.0,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,...,80.0,0,8,3,3,8,7,3,0,0
4,2.0,NaN,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,...,80.0,1,6,3,3,2,2,2,2,0
5,4.0,32.0,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,...,80.0,0,8,2,2,7,7,3,6,0
6,1.0,59.0,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,...,80.0,3,12,3,2,1,0,0,0,0
7,3.0,30.0,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,...,80.0,1,1,2,3,1,0,0,0,0
8,3.0,38.0,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,...,80.0,0,10,2,3,9,7,1,8,0
9,3.0,36.0,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,...,80.0,2,17,3,2,7,7,7,7,0


## **b. Limpieza y preparación de datos.**

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1506 entries, 0 to 1505
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   JobSatisfaction           1494 non-null   float64
 1   Age                       1503 non-null   float64
 2   Attrition                 1505 non-null   object 
 3   BusinessTravel            1506 non-null   object 
 4   DailyRate                 1506 non-null   int64  
 5   Department                1506 non-null   object 
 6   DistanceFromHome          1506 non-null   int64  
 7   Education                 1506 non-null   int64  
 8   EducationField            1506 non-null   object 
 9   EmployeeCount             1506 non-null   int64  
 10  EmployeeNumber            1506 non-null   int64  
 11  EnvironmentSatisfaction   1506 non-null   int64  
 12  Gender                    1506 non-null   object 
 13  HourlyRate                1506 non-null   int64  
 14  JobInvol

In [6]:
data.describe()

,JobSatisfaction,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,NewEmployee
count,1494.000000,1503.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,...,1499.0,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000
mean,2.744311,37.086494,803.978752,9.229084,2.915671,1.000664,753.500000,2.729084,66.280212,2.729084,...,80.0,0.790173,11.220452,2.802789,2.758964,7.041169,4.212483,2.169987,4.112882,0.007968
std,1.099938,10.848391,403.962331,8.135153,1.031172,0.025768,434.889066,1.092854,21.275971,0.717816,...,0.0,0.850911,7.778530,1.293076,0.705698,6.568092,3.618337,3.196186,3.572208,0.088938
min,1.000000,3.000000,102.000000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,1.000000,...,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,30.000000,465.000000,2.000000,2.000000,1.000000,377.250000,2.000000,48.000000,2.000000,...,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000,0.000000
50%,3.000000,36.000000,805.000000,7.000000,3.000000,1.000000,753.500000,3.000000,66.000000,3.000000,...,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000,0.000000
75%,4.000000,43.000000,1161.000000,14.000000,4.000000,1.000000,1129.750000,4.000000,84.000000,3.000000,...,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000,0.000000
max,4.000000,200.000000,1499.000000,29.000000,7.000000,2.000000,1506.000000,4.000000,190.000000,4.000000,...,80.0,3.000000,40.000000,6.000000,4.000000,100.000000,18.000000,15.000000,17.000000,1.000000


De una primera observación general de los datos se puede indentificar:
  * Se tienen 36 columnas de datos, y 1506 registros.
  * Se tienen 3 tipos de datos: float64(3), int64(24), object(9).
  * Las siguientes columnas tienen datos incompletos: JobSatisfaction(1494), Age(1503), Attrition(1505), y StandardHours (1499).

### JobSatisfaction

In [7]:
#datos vacíos columna JobSatisfaction
data[data['JobSatisfaction'].isnull()]['NewEmployee']

15      1
69      1
129     1
265     1
604     1
739     1
844     1
887     1
907     1
990     1
1161    1
1374    1
Name: NewEmployee, dtype: int64

Como datos vacíos se identificaron los 12 registros (NewEmployee = 1) para evaluar los modelos. Se procede a almacenar éstos datos en un nuevo dataframe para evaluar después, y por ahora quitarlos de los datos.

In [8]:
NewEmployee = data.copy()
NewEmployee = NewEmployee[NewEmployee['JobSatisfaction'].isnull()].drop(['JobSatisfaction'], axis=1)
NewEmployee

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,NewEmployee
15,29.0,No,Travel_Rarely,1389,Research & Development,21,4,Life Sciences,1,16,...,80.0,1,10,1,3,10,9,8,8,1
69,36.0,Yes,Travel_Rarely,318,Research & Development,9,3,Medical,1,70,...,80.0,1,2,0,2,1,0,0,0,1
129,49.0,No,Travel_Rarely,470,Research & Development,20,4,Medical,1,130,...,80.0,0,16,2,2,15,11,5,11,1
265,29.0,No,Travel_Rarely,1210,Sales,2,3,Medical,1,266,...,80.0,2,10,2,3,0,0,0,0,1
604,42.0,No,Travel_Rarely,933,Research & Development,29,3,Life Sciences,1,605,...,80.0,1,10,3,2,9,8,7,8,1
739,27.0,No,Travel_Rarely,1055,Research & Development,2,4,Life Sciences,1,740,...,80.0,1,4,2,3,3,2,2,2,1
844,30.0,No,Travel_Rarely,852,Sales,10,3,Marketing,1,845,...,80.0,1,10,3,3,10,3,1,4,1
887,42.0,No,Travel_Frequently,458,Research & Development,26,5,Medical,1,888,...,80.0,0,20,6,3,1,0,0,0,1
907,44.0,No,Travel_Rarely,1099,Sales,5,3,Marketing,1,908,...,80.0,1,26,5,3,22,9,3,10,1
990,33.0,No,Travel_Frequently,1111,Sales,5,1,Life Sciences,1,991,...,80.0,0,8,2,4,5,4,1,2,1
